### Implementation of Modern Portfolio Theory

In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
import datetime as dt
import scipy as sc

In [2]:
# Import data
def getData(stocks, start, end):
    stockData = yf.download(stocks, start, end)
    stockData = stockData['Close']
    
    returns = stockData.pct_change()
    meanReturns = returns.mean()
    covMatrix = returns.cov()

    return meanReturns, covMatrix

def portfolio_performance(weights, meanReturns, covMatrix):
    returns = np.sum(meanReturns*weights)*252
    std = np.sqrt(np.dot(weights.T, np.dot(covMatrix, weights))) * np.sqrt(252)
    return returns, std

stockList = ['CBA', 'BHP', "TLS"]
stocks = [stock+'.AX' for stock in stockList]

endDate = dt.datetime.now()
startDate = endDate - dt.timedelta(days=365)

weights = np.array([0.3, 0.3, 0.4])

In [3]:
def negative_sharpe_ratio(weights, meanReturns, covMatrix, riskFreeRate=0):
    pReturns, pStd = portfolio_performance(weights, meanReturns, covMatrix)
    return -(pReturns - riskFreeRate)/pStd

In [4]:
def maximize_sharpe_ratio(meanReturns, covMatrix, riskFreeRate = 0, constraintSet = (0,1)):
    "Minimize the negative sharpe ratio, by altering the weights of the portfolio"
    numAssets = len(meanReturns)
    args = (meanReturns, covMatrix, riskFreeRate)
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    bound = constraintSet
    bounds = tuple(bound for asset in range(numAssets))
    result = sc.optimize.minimize(negative_sharpe_ratio, numAssets*[1./numAssets], args=args, method='SLSQP', bounds=bounds, constraints=constraints)
    return result

In [5]:
meanReturns , covMatrix = getData(stocks, startDate, endDate)
returns, std = portfolio_performance(weights, meanReturns, covMatrix)
results = maximize_sharpe_ratio(meanReturns, covMatrix)
print(results)

[*********************100%%**********************]  3 of 3 completed

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: -0.8397760190358585
       x: [ 0.000e+00  1.000e+00  1.665e-16]
     nit: 2
     jac: [ 4.625e-01 -7.451e-09  1.353e+00]
    nfev: 8
    njev: 2


In [6]:
max_sharpe_ratio, maxWeights = results['fun'], results['x']
print(max_sharpe_ratio, maxWeights)

-0.8397760190358585 [0.00000000e+00 1.00000000e+00 1.66533454e-16]


In [20]:
def portfolio_variance(weights, meanReturns, covMatrix):
    return portfolio_performance(weights, meanReturns, covMatrix)[1]

In [23]:
def minimize_variance(meanReturns, covMatrix, constraintSet=(0,1)):
    "Minimize the portfolio variance by altering the weights/allocation of assets in the portfolio"
    numAssets = len(meanReturns)
    args = (meanReturns, covMatrix)
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    bound = constraintSet
    bounds = tuple(bound for asset in range(numAssets))
    result = sc.optimize.minimize(portfolio_variance, numAssets*[1./numAssets], args=args, method='SLSQP', bounds=bounds, constraints=constraints)
    return result

In [24]:
minVarResult = minimize_variance(meanReturns, covMatrix)
minVar, minVarWeights = minVarResult['fun'], minVarResult['x']
print(minVar, minVarWeights)

0.11036502307867817 [0.14012145 0.28819784 0.5716807 ]
